![Business Intelligence Banner](https://img.freepik.com/premium-vector/business-intelligence-implementation-word-concepts-dark-blue-banner_106317-24117.jpg?w=826)

Al entregar la solución de este parcial, yo, **David Mateo Barbosa Monsalve** con código **202110756** me comprometo a no conversar durante el desarrollo de este examen con ninguna persona que no sea el profesor del curso, sobre aspectos relacionados con el parcial; tampoco utilizaré algún medio de comunicación por voz, texto o intercambio de archivos, para consultar o compartir con otros, información sobre el tema del parcial. Ni usaré alguna IA para resolver las preguntas del parcial. Soy consciente y acepto las consecuencias que acarreará para mi desempeño académico cometer fraude en este parcial.